In [1]:
import matplotlib
import numpy as np
import scipy as scp
import pywt
import matplotlib.pyplot as plt
%matplotlib inline

import os
import pickle

import pdb

In [2]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from sklearn.pipeline import Pipeline

In [3]:
from transform_scatt import load_vertical_transform, Concat_scat_tree, Concat_scal, JointScat

In [4]:
from compute_features import compute_features, load_features
from plot_utils import plot_confusion_matrix

# Hyperparameters

In [5]:
db_location = '/users/data/blier/ESC-50/'
log_features = True
log_eps = 0.001

connections = 'pca_net'

#update this list to test over different features
features = 'mfcc' #, 'plain_scat_1', 'plain_scat_2']

params = {'channels': (84,12), 'hops': (512,4),
          'fmin':32.7, 'fmax':11001,
          'alphas':(6,6),'Qs':(12,12), # only used for flex scattering
          'nclasses': 50, 'n_itemsbyclass':40, 'max_sample_size':2**17,
          'audio_ext':'*.ogg'}

nOctaves=10
nfo=12
nfo2=12

## Load and prepare the Data

In [6]:
load = 'joint'
if load == 'scat':
    scat_trans = "/users/data/blier/features_esc50/scat/"
    X, y = load_features(scat_trans, params['nclasses'], params['n_itemsbyclass'])
elif load == 'raw':
    root_path = "/users/data/blier/ESC-50"
    X, y = compute_features(root_path, features, params)
elif load == 'joint':
    vert_wav = JointScat(nOctaves, nfo, nfo2)
    scat_trans = "/users/data/blier/features_esc50/scat_10_12_6/"
    X, y = load_vertical_transform(scat_trans, params['nclasses'], params['n_itemsbyclass'], vert_wav)
else:
    raise ValueError("Unknown loading parameter")

101 - Dog.pkl
shape (40, 637)
102 - Rooster.pkl
shape (40, 637)
103 - Pig.pkl
shape (40, 637)
104 - Cow.pkl
shape (40, 637)
105 - Frog.pkl
shape (40, 637)
106 - Cat.pkl
shape (40, 637)
107 - Hen.pkl
shape (40, 637)
108 - Insects.pkl
shape (40, 637)
109 - Sheep.pkl
shape (40, 637)
110 - Crow.pkl
shape (40, 637)
201 - Rain.pkl
shape (40, 637)
202 - Sea waves.pkl
shape (40, 637)
203 - Crackling fire.pkl
shape (40, 637)
204 - Crickets.pkl
shape (40, 637)
205 - Chirping birds.pkl
shape (40, 637)
206 - Water drops.pkl
shape (40, 637)
207 - Wind.pkl
shape (40, 637)
208 - Pouring water.pkl
shape (40, 637)
209 - Toilet flush.pkl
shape (40, 637)
210 - Thunderstorm.pkl
shape (40, 637)
301 - Crying baby.pkl
shape (40, 637)
302 - Sneezing.pkl
shape (40, 637)
303 - Clapping.pkl
shape (40, 637)
304 - Breathing.pkl
shape (40, 637)
305 - Coughing.pkl
shape (40, 637)
306 - Footsteps.pkl
shape (40, 637)
307 - Laughing.pkl
shape (40, 637)
308 - Brushing teeth.pkl
shape (40, 637)
309 - Snoring.pkl
shape (4

In [7]:
classifier1 = SVC(C=1., kernel='linear')
classifier2 = RandomForestClassifier()
#('vertical_cwt', Vertical_wavelet())
#estimators = [('concat', Concat_scat_tree(M = 3, transf='mean')), 
#              ('classifier', classifier1)]
#estimators = [('concat', Vertical_wavelet()), 
#              ('classifier', classifier1)]
estimators = [('classifier', classifier1)]
#estimators = [('concat', Concat_scal(transf='max')),('svc', SVC(C=1., kernel='linear'))]
pipe = Pipeline(estimators)

In [8]:
cross_valid = True
if cross_valid:
    scores = cross_val_score(pipe, X, y, cv=10)
    print(scores), scores.mean()
else:
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.25, 
                         random_state=42, stratify=y)
    pipe.fit(X_train, y_train)
    score_train = pipe.score(X_train, y_train)
    score_test = pipe.score(X_test, y_test)
    score_train, score_test

[ 0.705  0.69   0.735  0.68   0.675  0.74   0.755  0.725  0.715  0.68 ]


In [9]:
scores.mean()

0.70999999999999996

In [10]:
y_pred = pipe.predict(X_test)
conf = confusion_matrix(y_test, y_pred)
conf_surclasses = confusion_matrix(y_test // 10, y_pred // 10)

NameError: name 'X_test' is not defined

## Confusion Matrix

In [ ]:
plot_confusion_matrix(conf, range(params["nclasses"]), plot_values = False, normalize=True)

In [ ]:
plot_confusion_matrix(conf_surclasses, range(5), plot_values = True, normalize=True)